In [5]:
import xml.etree.ElementTree as ET
import pandas as pd

def xml_to_dataframe(xml_string):
    root = ET.fromstring(xml_string)
    data = []

    def parse_element(element, parent_path=""):
        path = parent_path + element.tag
        if element.attrib:
            path += "_" + "_".join([f"{k}_{v}" for k, v in element.attrib.items()])
        if element.text:
            data.append((path, element.text))
        for child in element:
            parse_element(child, parent_path=path + "_")

    parse_element(root)

    df = pd.DataFrame(data, columns=["Path", "Value"])
    df["Tag"] = df["Path"].apply(lambda x: x.split("_")[-1])
    df["Path"] = df["Path"].apply(lambda x: "_".join(x.split("_")[:-1]))
    df = df.groupby(["Path", "Tag"])["Value"].apply(lambda x: "|".join(x)).reset_index()
    df = df.pivot(index="Path", columns="Tag", values="Value").reset_index()

    return df

# Example XML string
xml_string = """
<root>
    <person>
        <name>John Doe</name>
        <age>30</age>
        <address>
            <street>123 Main St</street>
            <city>Anytown</city>
        </address>
    </person>
    <person>
        <name>Jane Smith</name>
        <age>25</age>
        <address>
            <street>456 Elm St</street>
            <city>Somecity</city>
        </address>
    </person>
</root>
"""

# Convert XML to DataFrame
df = xml_to_dataframe(xml_string)

# Print the DataFrame
print(df)


Tag                 Path                        address    age              city                 name                 person    root                  street
0                                                   NaN    NaN               NaN                  NaN                    NaN  \n                         NaN
1                   root                            NaN    NaN               NaN                  NaN  \n        |\n             NaN                     NaN
2            root_person  \n            |\n              30|25               NaN  John Doe|Jane Smith                    NaN     NaN                     NaN
3    root_person_address                            NaN    NaN  Anytown|Somecity                  NaN                    NaN     NaN  123 Main St|456 Elm St
